In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Utils

In [ ]:
#Utils
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, roc_auc_score, make_scorer, accuracy_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC 
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import StratifiedKFold
import lightgbm
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_uniform
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics.pairwise  import cosine_similarity
from sklearn.metrics.pairwise import chi2_kernel
from catboost import CatBoostClassifier
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
from sklearn.manifold import TSNE
from sklearn.ensemble import IsolationForest

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
!pip install -q -U matplotlib==3.1.3

In [ ]:
!pip install -q -U git+https://github.com/mljar/mljar-supervised.git@master

In [ ]:
from supervised.automl import AutoML

### Load Data

In [ ]:
train= pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv', sep=',')
sub_sample = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv', sep=',')
test= pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv', sep=',')

In [ ]:
train = train.set_index('id')

In [ ]:
test = test.set_index('id')

In [ ]:
fig = px.histogram(train, x="target",
                   width=600, 
                   height=400,
                   histnorm='percent',
                   template="simple_white"
                   )
fig.update_layout(title="Target Description", 
                  font_family="San Serif",
                  titlefont={'size': 20},
                  showlegend=True,
                  legend=dict(
                      orientation="v", 
                      y=1, 
                      yanchor="top", 
                      x=1.0, 
                      xanchor="right"
                  )                
                 ).update_xaxes(categoryorder='total descending')#
fig.update_traces( 
                  marker_line_width=1.5, opacity=0.99)
fig.show()

In [ ]:
cols = train.columns[:-1] 
df1= train[train['target']==train.target.unique()[0]]
df2= train[train['target']==train.target.unique()[1]]
df3= train[train['target']==train.target.unique()[2]]
df4= train[train['target']==train.target.unique()[3]]

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(y=df1[cols].mean(),
                         x=cols,
                         name = str(train.target.unique()[0]),
                         line=dict(color='royalblue', width=2, dash='solid')
                         ))
fig.add_trace(go.Scatter(y=df2[cols].mean(),
                         x=cols,
                         name = str(train.target.unique()[1]),
                         line=dict(color='red', width=2, dash='solid')
                         ))
fig.add_trace(go.Scatter(y=df3[cols].mean(),
                         x=cols,
                         name = str(train.target.unique()[2]),
                         line=dict(color='orange', width=2, dash='solid')
                         ))
fig.add_trace(go.Scatter(y=df4[cols].mean(),
                         x=cols,
                         name = str(train.target.unique()[3]),
                         line=dict(color='green', width=2, dash='solid')
                         ))


fig.update_layout(title='<b>Average value by Class<b>',
                  xaxis_title='Features',
                  yaxis_title='Average Value',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=900, height=500)

fig.update_yaxes(title_text="<b>values</b>", secondary_y=True)

fig.show()    

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(y=df1[cols].std(),
                         x=cols,
                         name = str(train.target.unique()[0]),
                         line=dict(color='royalblue', width=2, dash='solid')
                         ))
fig.add_trace(go.Scatter(y=df2[cols].std(),
                         x=cols,
                         name = str(train.target.unique()[1]),
                         line=dict(color='red', width=2, dash='solid')
                         ))
fig.add_trace(go.Scatter(y=df3[cols].std(),
                         x=cols,
                         name = str(train.target.unique()[2]),
                         line=dict(color='orange', width=2, dash='solid')
                         ))
fig.add_trace(go.Scatter(y=df4[cols].std(),
                         x=cols,
                         name = str(train.target.unique()[3]),
                         line=dict(color='green', width=2, dash='solid')
                         ))


fig.update_layout(title='<b>Std.Dev by value by Class<b>',
                  xaxis_title='Features',
                  yaxis_title='Average Value',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=900, height=500)

fig.update_yaxes(title_text="<b>values</b>", secondary_y=True)

fig.show()    

In [ ]:
train['sum_row'] = train[cols].sum(axis=1)
fig = px.box(train,x='target', y="sum_row")

fig.update_layout(title='<b>Sum row wise by class<b>',
                  xaxis_title='Class',
                  yaxis_title='Sum of features',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=900, height=500)

fig.show()
train = train.drop(columns=['sum_row'])

In [ ]:
fig3 = make_subplots(specs=[[{"secondary_y": True}]])
fig3.add_trace(go.Scatter(y=(train[cols]<0).sum(),
                         x=cols,
                         name = str(train.target.unique()[0]),
                         line=dict(color='royalblue', width=2, dash='solid')
                         ))
fig3.update_layout(title='<b>#Negative istances by feature in training set<b>',
                  xaxis_title='Feature',
                  yaxis_title='#Negatives',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=1000, height=500)
fig3.show()

In [ ]:
fig3 = make_subplots(specs=[[{"secondary_y": True}]])
fig3.add_trace(go.Scatter(y=(test[cols]<0).sum(),
                         x=cols,
                         name = '#negative',
                         line=dict(color='green', width=2, dash='solid')
                         ))
fig3.update_layout(title='<b>#Negative istances by feature in test set<b>',
                  xaxis_title='Feature',
                  yaxis_title='#Negatives',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=1000, height=500)
fig3.show()

In [ ]:
df = pd.concat([train[cols], test[cols]], axis=0)

In [ ]:
unique_df = pd.DataFrame(df.nunique()).reset_index()
unique_df.columns=['features','count']

fig1 = px.bar(unique_df, y='count', x=cols)

fig1.update_layout(title='Feature cardinality in train+test set',
                  xaxis_title='features',
                  yaxis_title='# unique values',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=900, height=500)
fig1.show()

In [ ]:
fig3 = make_subplots(specs=[[{"secondary_y": True}]])
fig3.add_trace(go.Scatter(y=(train[cols]==0).mean(),
                         x=cols,
                         name = str(train.target.unique()[0]),
                         line=dict(color='royalblue', width=2, dash='solid')
                         ))
fig3.update_layout(title='<b>% features=0 in training Set set<b>',
                  xaxis_title='Feature',
                  yaxis_title='#Negatives',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=1000, height=500)
fig3.show()

In [ ]:
fig3 = make_subplots(specs=[[{"secondary_y": True}]])
fig3.add_trace(go.Scatter(y=(test[cols]==0).mean(),
                         x=cols,
                         name = str(train.target.unique()[0]),
                         line=dict(color='green', width=2, dash='solid')
                         ))
fig3.update_layout(title='<b>% features=0 in test Set set<b>',
                  xaxis_title='Feature',
                  yaxis_title='#Negatives',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=1000, height=500)
fig3.show()

In [ ]:
corr = train[train.columns[:-2]].corr()
fig = go.Figure(data= go.Heatmap(z=corr,
                                 x=corr.index.values,
                                 y=corr.columns.values,
                                 zmin=-0.05,
                                 zmax=0.05
                                 )
                )
fig.update_layout(title_text='<b>Correlation Matrix<b>',
                  title_x=0.5,
                  titlefont={'size': 24},
                  width=900, height=800,
                  xaxis_showgrid=False,
                  yaxis_showgrid=False,
                  yaxis_autorange='reversed', 
                  paper_bgcolor=None,
                  )
fig.show()

In [ ]:
clf=IsolationForest(n_estimators=100, max_samples='auto', contamination=0.001, \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=22, verbose=0)
clf.fit(train[cols])
pred = clf.predict(train[cols])
train['anomaly']=pred
outliers=train.loc[train['anomaly']==-1]
outlier_index=list(outliers.index)

In [ ]:
pca = PCA(2)
pca.fit(train[cols])
res=pd.DataFrame(pca.transform(train[cols]))
Z = np.array(res)

In [ ]:
fig2 = go.Figure()

# Add traces
fig2.add_trace(go.Scatter(x=res[0], y=res[1],
                    mode='markers',
                    name='Normal'))

fig2.add_trace(go.Scatter(x=res.iloc[outlier_index,0], y=res.iloc[outlier_index,1],
                    mode='markers',
                    name='Predicted Outliers'))

fig2.update_layout(title='Isolation Forest',
                  xaxis_title='pca1',
                  yaxis_title='pca2',
                  titlefont={'size': 28, 'family':'Serif'},
                  template='simple_white',
                  showlegend=True,
                  width=900, height=500)

fig2.show()

In [ ]:
train= pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv', sep=',')
test= pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv', sep=',')
train = train.set_index('id')
test = test.set_index('id')

In [ ]:
cols = train.columns[:-1]
target = train['target']
train = train[cols]

In [ ]:
# create train and test sets
X_train, X_val, y_train, y_val = train_test_split(train, target, train_size=0.75, test_size=0.25, stratify=target, random_state=2)

In [ ]:
# train using Explain
automl = AutoML(mode="Explain",ml_task = 'multiclass_classification',eval_metric='logloss', explain_level=2)
automl.fit(X_train, y_train)

In [ ]:
automl.report()